Define label name and feature names

In [ ]:
import pandas as pd
import numpy as np

LBL = "label_2" # Speaker age
LABELS = ['label_1','label_2', 'label_3', 'label_4']
FEATURES = [f'feature_{i}' for i in range(1, 257)]


Read training,validation and test data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


train_df = pd.read_csv("/content/drive/MyDrive/ML_lab1/train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/ML_lab1/valid.csv")
test_df = pd.read_csv("/content/drive/MyDrive/ML_lab1/test.csv")


Mounted at /content/drive


Initialize dictionaries to store data

In [ ]:
train_x = {}
valid_x = {}
test_x = {}
train_y = {}
valid_y = {}
test_y = {}

Prepare and preprocess the data
Here, rows with missing values in label 2 have been dropped.

In [ ]:
from sklearn.preprocessing import RobustScaler

train_df_new = train_df[train_df['label_2'].notna()]# dropping rows with miising values for label 2 column.
valid_df_new = valid_df[train_df['label_2'].notna()]# dropping rows with miising values for label 2 column.
scaler = RobustScaler()
train_x[LBL] = pd.DataFrame(scaler.fit_transform(train_df_new.drop(LABELS, axis=1)), columns = FEATURES)
train_y[LBL] = train_df_new[LBL]
valid_x[LBL] = pd.DataFrame(scaler.transform(valid_df_new.drop(LABELS, axis=1)), columns = FEATURES)
valid_y[LBL] = valid_df_new[LBL]
test_x[LBL] = pd.DataFrame(scaler.transform(test_df.drop(LABELS, axis=1)), columns=FEATURES)
test_y[LBL] = test_df[LBL]

<ipython-input-4-ce03549e5df5>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_df_new = valid_df[train_df['label_2'].notna()]# dropping rows with miising values for label 2 column.


# For Label 2

Train K-Neighbors Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors=5)
regressor.fit(train_x[LBL], train_y[LBL])

KNeighborsRegressor()

Evaluate performance of K-Neighbors Regressor on the validation dataset

In [ ]:
from sklearn import metrics

y_predict_valid = regressor.predict(valid_x[LBL])

print("K-Neighbors Regressor Evaluation on Validation Set:")
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(valid_y[LBL], y_predict_valid)))



K-Neighbors Regressor Evaluation on Validation Set:
Root Mean Squared Error: 0.6446359868604573


Predict lable 1 for test dataset  using SVM classifier

In [ ]:
y_predict_test_before = regressor.predict(test_x[LBL])


## Applying Feature Engineering techniques

### Using SelectKBest

Apply SelectKBest on original features

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=190)
x_selected = selector.fit_transform(train_x[LBL], train_y[LBL])

Apply PCA on the SelectKBest-selected features

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95, svd_solver='full')
pca.fit(x_selected)

PCA(n_components=0.95, svd_solver='full')

In [ ]:
train_x_trans = pd.DataFrame(pca.transform(x_selected))
valid_x_trans = pd.DataFrame(pca.transform(selector.transform(valid_x[LBL])))
test_x_trans = pd.DataFrame(pca.transform(selector.transform(test_x[LBL])))

print("Shape after feature reduction:", train_x_trans.shape)

Shape after feature reduction: (28040, 62)


Train the K-Neighbors Regressor on PCA-transformed SelectKBest-selected features


In [ ]:
regressor = KNeighborsRegressor(n_neighbors=5)
regressor.fit(train_x_trans, train_y[LBL])

KNeighborsRegressor()

Evaluate performance of K-Neighbors Regressor on PCA-transformed SelectKBest-selected features using validation dataset




In [ ]:
y_predict_valid_pca = regressor.predict(valid_x_trans)

In [ ]:
print("K-Neighbors Regressor Evaluation on PCA-transformed SelectKBest-selected features (Validation Set):")
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(valid_y[LBL], y_predict_valid_pca)))


K-Neighbors Regressor Evaluation on PCA-transformed SelectKBest-selected features (Validation Set):
Root Mean Squared Error: 0.6977211583539765


Predict lable 1 for test dataset after feature reduction

In [ ]:
y_predict_test_after = regressor.predict(test_x_trans)

In [ ]:
output_df = pd.DataFrame({
    'Predicted labels before feature engineering': y_predict_test_before,
    'Predicted labels after feature engineering': y_predict_test_after,
    'No of new features': test_x_trans.shape[1]
})


for i in range(test_x_trans.shape[1]):
    output_df[f'new_feature_{i+1}'] = test_x_trans.iloc[:, i]
# Create a list of new column names
new_columns = [f'new_feature_{i+1}' for i in range(test_x_trans.shape[1], 256)]

# Concatenate the new columns to the existing DataFrame
output_df = pd.concat([output_df, pd.DataFrame(columns=new_columns)], axis=1)


In [ ]:
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/files/190334K_label_2.csv', index=False)